In [2]:
import numpy as np
import os
import urllib
import re
import bs4
import pandas as pd
import datetime
import tempfile
import zipfile
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df_text_versions = pd.read_csv("../raw_data/deep_course_cf_text_versions.csv", low_memory = False)
df_text_versions_node = pd.read_csv("../raw_data/deep_course_cf_text_version_nodes.csv", low_memory = False)
df_themes = pd.read_csv("../raw_data/deep_course_cf_themes.csv", low_memory = False)
df_types = pd.read_csv("../raw_data/deep_course_cf_types.csv", low_memory = False)
df_nodes = pd.read_csv("../raw_data/deep_course_cm_nodes.csv", low_memory = False)

df_formulaire_question = pd.read_csv("../raw_data/deep_course_formulaire_question.csv", low_memory = False)
df_formulaire_questionnaire = pd.read_csv("../raw_data/deep_course_formulaire_questionnaire.csv", low_memory = False)
df_formulaire_questionnaire_pages = pd.read_csv("../raw_data/deep_course_formulaire_questionnaire_pages.csv", low_memory = False)
df_formulaire_question_choix = pd.read_csv("../raw_data/deep_course_formulaire_question_choix.csv", low_memory = False)
df_formulaire_reponse = pd.read_csv("../raw_data/deep_course_formulaire_reponse.csv", low_memory = False)
df_formulaire_page_questions = pd.read_csv("../raw_data/deep_course_formulaire_page_questions.csv", low_memory = False)

## Base texte

In [4]:
df_themes.columns = df_themes.columns.str.replace('id','theme_id')
df = pd.merge(df_text_versions, df_themes, on='theme_id')

df_types.columns = df_types.columns.str.replace('id','type_id')
df_text = pd.merge(df, df_types, on="type_id")
df_text

,surrogate_uuid,group_id,txt_name,theme_id,type_id,domain,theme,applicabilité
0,2c90f588404b484801404d8600d87748,TXA9548,RÈGLEMENT (UE) N°715/2013 DE LA COMMISSION du ...,4028807d16245e4d01162524c71b6a4d,4028807d16245e4d01162524ed896c33,1. ENVIRONNEMENT,Déchets,Applicable
1,2c90f58846fdd49601476d7519a334ea,TXA6524,Règlement (CE) n° 1418/2007 du 29 novembre 200...,4028807d16245e4d01162524c71b6a4d,4028807d16245e4d01162524ed896c33,1. ENVIRONNEMENT,Déchets,Applicable
2,2c90f589515fb45c01516335125951f6,TXA5166,Règlement (CE) n° 1013/2006 du Parlement europ...,4028807d16245e4d01162524c71b6a4d,4028807d16245e4d01162524ed896c33,1. ENVIRONNEMENT,Déchets,Applicable
3,4028807d16245e4d0116254356975904,TXA3451,Règlement du Conseil n° 259/93 du 1er février ...,4028807d16245e4d01162524c71b6a4d,4028807d16245e4d01162524ed896c33,1. ENVIRONNEMENT,Déchets,Applicable
4,4028807d16245e4d01162546ca9d716c,TXA6370,Règlement n° 574/2004 de la Commission du 23 f...,4028807d16245e4d01162524c71b6a4d,4028807d16245e4d01162524ed896c33,1. ENVIRONNEMENT,Déchets,Applicable
5,d37394453bb51e7d013bbc9581311dbc,TXA9353,Règlement (UE) no 1179/2012 de la Commission d...,4028807d16245e4d01162524c71b6a4d,4028807d16245e4d01162524ed896c33,1. ENVIRONNEMENT,Déchets,Applicable
6,d37394c52e599cde012ec2cf221b3c45,TXA6956,Règlement européen (CE) n° 1774/2002 du 3 octo...,4028807d16245e4d01162524c71b6a4d,4028807d16245e4d01162524ed896c33,1. ENVIRONNEMENT,Déchets,Applicable
7,d37394c52fc2d231012fc5ca47ad4755,TXA8836,RÈGLEMENT (UE) N o 333/2011 DU CONSEIL du 31 m...,4028807d16245e4d01162524c71b6a4d,4028807d16245e4d01162524ed896c33,1. ENVIRONNEMENT,Déchets,Applicable
8,d37394d916f7c2060116fd354b7d06a5,TXA6268,Règlement (CE) n° 801/2007 de la Commission du...,4028807d16245e4d01162524c71b6a4d,4028807d16245e4d01162524ed896c33,1. ENVIRONNEMENT,Déchets,Applicable
9,d37394d918f9a5c501190fa862aa1643,TXA6904,Règlement du Conseil no 1420/1999 du 29 avril ...,4028807d16245e4d01162524c71b6a4d,4028807d16245e4d01162524ed896c33,1. ENVIRONNEMENT,Déchets,Applicable


## Base client 

In [5]:
df_appliques = pd.merge(df_text_versions_node, df_text, how='left', left_on = ['txt_version_surrogate_uuid'], right_on = ['surrogate_uuid'])

In [5]:
df_appliques.head()

,txt_version_surrogate_uuid,txt_node_id,surrogate_uuid,group_id,txt_name,theme_id,type_id,domain,theme,applicabilité
0,d37394453d6ac204013d6d847b0a4b3a,2c90f5883f44af68013f564a1326558a,d37394453d6ac204013d6d847b0a4b3a,TXA9448,Décision BSEI n° 2012-150 du 17 décembre 2012 ...,ff8081816b4d71ef016b54dd71e53d99,4028807d16245e4d01162524e20d6b91,1. ENVIRONNEMENT,Canalisations de transport,Applicable
1,d37394453a046068013a0c21f6824adc,2c90f5883f44af68013f564a1326558a,d37394453a046068013a0c21f6824adc,TXA9312,Avis relatif à l’analyse de la régularité des ...,ff8081816b4d71ef016b54e0b788416d,4028807d16245e4d01162524e20d6b91,1. ENVIRONNEMENT,Travaux à proximité des réseaux / Guichet unique,Applicable
2,d37394453c64588f013c70d1f1b31b49,2c90f5883f44af68013f564a1326558a,d37394453c64588f013c70d1f1b31b49,TXA9229,Arrêté du 12 juin 2012 portant agrément des la...,4028807d16245e4d01162524c4c46a32,d37394c5272f761f01273d899b5811d1,1. ENVIRONNEMENT,Pollution atmosphérique,Pour information
3,d3739445374d5348013783ef84d41b5c,2c90f5883f44af68013f564a1326558a,d3739445374d5348013783ef84d41b5c,TXA9212,Arrêté du 24 avril 2012 fixant les caractérist...,4028807d16245e4d01162524c9ec6a71,4028807d16245e4d01162524db946b37,1. ENVIRONNEMENT,Installations classées,Applicable
4,d373944539678fdb0139a42ace673d21,2c90f5883f44af68013f564a1326558a,d373944539678fdb0139a42ace673d21,TXA9136,Arrêté du 29 février 2012 fixant le contenu de...,4028807d16245e4d01162524c71b6a4d,4028807d16245e4d01162524db946b37,1. ENVIRONNEMENT,Déchets,Applicable


In [6]:
df_appliques_clean=df_appliques.dropna(axis=0)

In [7]:
df_appliques.shape

(329942, 10)

In [8]:
df_appliques_clean.groupby("surrogate_uuid")["surrogate_uuid"].count().sort_values(ascending= True).value_counts().head(20)

1     316
6     131
2     130
3     101
4      86
9      75
5      74
7      62
16     48
18     46
17     45
19     45
10     43
8      39
12     37
29     36
28     33
33     32
20     30
15     30
Name: surrogate_uuid, dtype: int64

In [9]:
iden=df_appliques_clean["surrogate_uuid"].drop_duplicates(keep = 'first').sort_values().reset_index(drop=True)

In [10]:
valeur=pd.Series((df_appliques_clean.groupby("surrogate_uuid")["surrogate_uuid"].count()==1).values)

In [11]:
mixte=pd.concat([iden,valeur],axis=1)
mixte

,surrogate_uuid,0
0,2c90f588404b484801404d8600d87748,False
1,2c90f588404b484801404dce49f11efc,False
2,2c90f588404b484801404df98f7140d7,False
3,2c90f588404b484801404dfacbff443e,False
4,2c90f588404b484801404e0154db49b0,False
5,2c90f588404b484801404e0553614a10,False
6,2c90f588404b484801404e5f94ee6d3f,False
7,2c90f588404b48480140588b113b2317,False
8,2c90f588404b4848014058a6510a2eb4,False
9,2c90f588404b4848014058c7b21752ab,False


In [12]:
liste=mixte.loc[mixte.iloc[:,1]==True,'surrogate_uuid'].values

In [13]:
df_appliques_clean2=df_appliques_clean.drop(df_appliques_clean.loc[df_appliques_clean["surrogate_uuid"].isin(liste)].index)

In [14]:
df_appliques_clean2.groupby("surrogate_uuid")["surrogate_uuid"].count().sort_values(ascending= True).value_counts().head(20)

6     131
2     130
3     101
4      86
9      75
5      74
7      62
16     48
18     46
17     45
19     45
10     43
8      39
12     37
29     36
28     33
33     32
15     30
20     30
14     29
Name: surrogate_uuid, dtype: int64

In [15]:
base_client_texte=df_appliques_clean2[['txt_node_id','surrogate_uuid']]

In [16]:
base_client_texte.shape

(322190, 2)

In [17]:
base_client_texte.reset_index(drop=True)

,txt_node_id,surrogate_uuid
0,2c90f5883f44af68013f564a1326558a,d37394453d6ac204013d6d847b0a4b3a
1,2c90f5883f44af68013f564a1326558a,d37394453a046068013a0c21f6824adc
2,2c90f5883f44af68013f564a1326558a,d37394453c64588f013c70d1f1b31b49
3,2c90f5883f44af68013f564a1326558a,d3739445374d5348013783ef84d41b5c
4,2c90f5883f44af68013f564a1326558a,d373944539678fdb0139a42ace673d21
5,2c90f5883f44af68013f564a1326558a,d373944535c610f20135ec1883a854bf
6,2c90f5883f44af68013f564a1326558a,d3739445355c87aa01355cae7693349d
7,2c90f5883f44af68013f564a1326558a,d373944534b518620134c1bbafe61cbd
8,2c90f5883f44af68013f564a1326558a,d37394c530e10683013109cace9c0673
9,2c90f5883f44af68013f564a1326558a,d37394c530e106830130fb4aaa1c307c


In [18]:
base_client_texte.describe()

,txt_node_id,surrogate_uuid
count,322190,322190
unique,572,3199
top,4028807d16245e4d0116249e4259205e,2c90f58864274849016427cf71e44af4
freq,1631,555


In [18]:
nom=base_client_texte['surrogate_uuid'].drop_duplicates(keep = 'first').sort_values(ascending= True).values

In [19]:
nom

array(['2c90f588404b484801404d8600d87748',
       '2c90f588404b484801404dce49f11efc',
       '2c90f588404b484801404df98f7140d7', ...,
       'ff80818174e6040901750258b1891163',
       'ff80818174e60409017502da6e6d1ec6',
       'ff808181755a45d801757a80417525ad'], dtype=object)

In [20]:
test=pd.get_dummies( data=base_client_texte, columns=['surrogate_uuid'])

In [21]:
test.columns

Index(['txt_node_id', 'surrogate_uuid_2c90f588404b484801404d8600d87748',
       'surrogate_uuid_2c90f588404b484801404dce49f11efc',
       'surrogate_uuid_2c90f588404b484801404df98f7140d7',
       'surrogate_uuid_2c90f588404b484801404dfacbff443e',
       'surrogate_uuid_2c90f588404b484801404e0154db49b0',
       'surrogate_uuid_2c90f588404b484801404e0553614a10',
       'surrogate_uuid_2c90f588404b484801404e5f94ee6d3f',
       'surrogate_uuid_2c90f588404b48480140588b113b2317',
       'surrogate_uuid_2c90f588404b4848014058a6510a2eb4',
       ...
       'surrogate_uuid_ff80818174e604090174fcd0e0e1372d',
       'surrogate_uuid_ff80818174e60409017501c7d3183828',
       'surrogate_uuid_ff80818174e604090175022423814ddb',
       'surrogate_uuid_ff80818174e6040901750243646351e0',
       'surrogate_uuid_ff80818174e604090175024a97fa635a',
       'surrogate_uuid_ff80818174e604090175024d98626791',
       'surrogate_uuid_ff80818174e6040901750257377b0eed',
       'surrogate_uuid_ff80818174e604090175025

In [22]:
for i in range (1, len(test.columns)):
    test[test.columns[i]].astype('int32',copy=False)

In [23]:
test.shape

(322190, 3200)

In [24]:
test2=test.groupby('txt_node_id').apply(np.sum).drop('txt_node_id',axis=1)

In [25]:
test2

,surrogate_uuid_2c90f588404b484801404d8600d87748,surrogate_uuid_2c90f588404b484801404dce49f11efc,surrogate_uuid_2c90f588404b484801404df98f7140d7,surrogate_uuid_2c90f588404b484801404dfacbff443e,surrogate_uuid_2c90f588404b484801404e0154db49b0,surrogate_uuid_2c90f588404b484801404e0553614a10,surrogate_uuid_2c90f588404b484801404e5f94ee6d3f,surrogate_uuid_2c90f588404b48480140588b113b2317,surrogate_uuid_2c90f588404b4848014058a6510a2eb4,surrogate_uuid_2c90f588404b4848014058c7b21752ab,...,surrogate_uuid_ff80818174e604090174fcd0e0e1372d,surrogate_uuid_ff80818174e60409017501c7d3183828,surrogate_uuid_ff80818174e604090175022423814ddb,surrogate_uuid_ff80818174e6040901750243646351e0,surrogate_uuid_ff80818174e604090175024a97fa635a,surrogate_uuid_ff80818174e604090175024d98626791,surrogate_uuid_ff80818174e6040901750257377b0eed,surrogate_uuid_ff80818174e6040901750258b1891163,surrogate_uuid_ff80818174e60409017502da6e6d1ec6,surrogate_uuid_ff808181755a45d801757a80417525ad
txt_node_id,,,,,,,,,,,,,,,,,,,,,
2c90f5883f44af68013f564a1326558a,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2c90f5883fcf3281013fe7c2f0eb4dbe,0,1,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,0,0,1,1
2c90f588404b48480140ce4d7d301013,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2c90f5884109d2c401410c40ae942d95,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
2c90f5884109d2c401410c4434a433a6,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
2c90f5884109d2c401410c44a3493440,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
2c90f588429b4a8a01429f3055457934,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2c90f588429b4a8a01429f30cc007a32,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2c90f58845004b4601451c44694220d7,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [27]:
test2.to_pickle("../data/dummies_text.pkl")

## clustering

In [ ]:
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering

In [ ]:
model = AgglomerativeClustering(n_clusters=4,  )

model = model.fit(test2)

In [ ]:
model.labels_

In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

plt.title('Hierarchical Clustering Dendrogram')
# plot the top three levels of the dendrogram
plot_dendrogram(model)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

In [ ]:
import scipy.cluster.hierarchy as shc

plt.figure(figsize=(10, 7))
plt.title("Customer Dendograms")
dend = shc.dendrogram(shc.linkage(test2, method='ward'))

In [ ]:
#plt.figure(figsize=(10, 7))
#plt.title("Customer Dendograms")
#dend = shc.dendrogram(shc.linkage(test2, method='average'))

In [ ]:
test2

In [ ]:
test2.columns=nom

In [ ]:
res=test2.copy()

In [ ]:
res['clust']=model.labels_

In [ ]:
res

In [ ]:
res['clust'].value_counts()

In [ ]:
res.to_csv(r'C:\Users\Axel\Desktop\projet stat\res_clust.csv')

# Analyse

In [ ]:
res= pd.read_csv(r'C:\Users\Axel\Desktop\projet stat\res_clust.csv')
test2=pd.read_csv(r'C:\Users\Axel\Desktop\projet stat\clust.csv')

In [ ]:
res

In [ ]:
info_text=df_text[['surrogate_uuid','domain','theme']]

In [ ]:
base=base_client_texte.copy()

In [ ]:
base.head()

### On merge

In [ ]:
fus=pd.merge(base,info_text, how='left', on='surrogate_uuid')
fus.head()

In [ ]:
fus.to_pickle("fusion.pkl")

In [ ]:
res[['txt_node_id','clust']]

In [ ]:
fus2=pd.merge(fus,res[['txt_node_id','clust']],how='left', on='txt_node_id')
fus2.head()

## Analyse sur les domaines

### cluster 0

In [ ]:
#Nombre de lignes sur le cluster 0
fus2.loc[fus2['clust']==0]['txt_node_id'].drop_duplicates(keep = 'first').count()

In [ ]:
#Pour chaque client, nombre de fois qu'un domaine apparait divisé par le nombre de texte du client
table=(fus2.loc[fus2['clust']==0].groupby('txt_node_id')['domain'].value_counts()/fus2.loc[fus2['clust']==0].groupby('txt_node_id')['domain'].count())*100

In [ ]:
table.groupby('domain').mean()

In [ ]:
table.groupby('domain').sum()/fus2.loc[fus2['clust']==0]['txt_node_id'].drop_duplicates(keep = 'first').count()

### cluster 1

In [ ]:
fus2.loc[fus2['clust']==1]['txt_node_id'].drop_duplicates(keep = 'first').count()

In [ ]:
table=(fus2.loc[fus2['clust']==1].groupby('txt_node_id')['domain'].value_counts()/fus2.loc[fus2['clust']==1].groupby('txt_node_id')['domain'].count())*100
table.groupby('domain').mean()

In [ ]:
table.groupby('domain').sum()/fus2.loc[fus2['clust']==1]['txt_node_id'].drop_duplicates(keep = 'first').count()

### cluster 2

In [ ]:
fus2.loc[fus2['clust']==2]['txt_node_id'].drop_duplicates(keep = 'first').count()

In [ ]:
table=(fus2.loc[fus2['clust']==2].groupby('txt_node_id')['domain'].value_counts()/fus2.loc[fus2['clust']==2].groupby('txt_node_id')['domain'].count())*100
table.groupby('domain').mean()

In [ ]:
table.groupby('domain').sum()/fus2.loc[fus2['clust']==2]['txt_node_id'].drop_duplicates(keep = 'first').count()

### cluster 3

In [ ]:
fus2.loc[fus2['clust']==3]['txt_node_id'].drop_duplicates(keep = 'first').count()

In [ ]:
table=(fus2.loc[fus2['clust']==3].groupby('txt_node_id')['domain'].value_counts()/fus2.loc[fus2['clust']==3].groupby('txt_node_id')['domain'].count())*100
table.groupby('domain').mean()

In [ ]:
table.groupby('domain').sum()/fus2.loc[fus2['clust']==3]['txt_node_id'].drop_duplicates(keep = 'first').count()

### general

In [ ]:
fus2['txt_node_id'].drop_duplicates(keep = 'first').count()

In [ ]:
table=(fus2.groupby('txt_node_id')['domain'].value_counts()/fus2.groupby('txt_node_id')['domain'].count())*100
table.groupby('domain').mean()

In [ ]:
table.groupby('domain').sum()/fus2['txt_node_id'].drop_duplicates(keep = 'first').count()

## Analyse sur les thèmes

### cluster 0

In [ ]:
table=(fus2.loc[fus2['clust']==0].groupby('txt_node_id')['theme'].value_counts()/fus2.loc[fus2['clust']==0].groupby('txt_node_id')['theme'].count())*100
table.groupby('theme').mean().sort_values(ascending=False).head(10)

In [ ]:
(table.groupby('theme').sum()/fus2.loc[fus2['clust']==0]['txt_node_id'].drop_duplicates(keep = 'first').count()).sort_values(ascending=False).head(10)

### cluster 1

In [ ]:
table=(fus2.loc[fus2['clust']==1].groupby('txt_node_id')['theme'].value_counts()/fus2.loc[fus2['clust']==1].groupby('txt_node_id')['theme'].count())*100
table.groupby('theme').mean().sort_values(ascending=False).head(10)

In [ ]:
(table.groupby('theme').sum()/fus2.loc[fus2['clust']==1]['txt_node_id'].drop_duplicates(keep = 'first').count()).sort_values(ascending=False).head(10)

### cluster 2

In [ ]:
table=(fus2.loc[fus2['clust']==2].groupby('txt_node_id')['theme'].value_counts()/fus2.loc[fus2['clust']==2].groupby('txt_node_id')['theme'].count())*100
table.groupby('theme').mean().sort_values(ascending=False).head(10)

In [ ]:
(table.groupby('theme').sum()/fus2.loc[fus2['clust']==2]['txt_node_id'].drop_duplicates(keep = 'first').count()).sort_values(ascending=False).head(10)

### cluster 3

In [ ]:
table=(fus2.loc[fus2['clust']==3].groupby('txt_node_id')['theme'].value_counts()/fus2.loc[fus2['clust']==3].groupby('txt_node_id')['theme'].count())*100
table.groupby('theme').mean().sort_values(ascending=False).head(10)

In [ ]:
(table.groupby('theme').sum()/fus2.loc[fus2['clust']==3]['txt_node_id'].drop_duplicates(keep = 'first').count()).sort_values(ascending=False).head(10)

### general

In [ ]:
table=(fus2.groupby('txt_node_id')['theme'].value_counts()/fus2.groupby('txt_node_id')['theme'].count())*100
table.groupby('theme').mean().sort_values(ascending=False).head(10)

In [ ]:
(table.groupby('theme').sum()/fus2['txt_node_id'].drop_duplicates(keep = 'first').count()).sort_values(ascending=False).head(10)